# Konvolutivne neuronske mreže

Materijali su zasnovani na zvaničnom TensorFlow [tutorial](https://www.tensorflow.org/tutorials/images/cnn)-u.

Ovaj primer prikazuje konstrukciju, obučavanje i korišćenje konvolutivne
neuronske mreže (eng. convolutional neural network, cnn) za klasifikaciju
slika iz skupa podataka [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html).

### Import TensorFlow

In [0]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

### Skup podataka CIFAR10

Skup podataka `CIFAR10` sadrži 60,000 slika u boji koje su podeljene u 10
klasa, sa 6,000 slika u svakoj klasi. Skup je podeljen u 50,000 slika za
obučavanje i 10,000 slika za testiranje.

In [0]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalizujemo piksele na slikama
train_images, test_images = train_images / 255.0, test_images / 255.0

In [0]:
train_images[0].shape

Pogledajmo kako izgledaju neke slike u skupu, na primer prvih 25 slika.


In [0]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

### Definicija modela

U nastavku ćemo definisati prvi deo našeg modela. U ovom delu ćemo definisati
slojeve [konvolucije](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) i [agregacije](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) koji za cilj imaju da nauče novu
reprezentaciju podataka. Pomoću nove reprezentacije podataka ćemo kasnije
da olakšamo posao klasifikacionom modelu (drugi deo našeg modela).

Kao ulaz, mreža će uzeti ulaz dimenzije (visina slike, širina slike, broj kanala) uz dodatnu dimenziju
koja se koristi za veličinu podskupa (eng. batch size).

Broj kanala će biti 3 usled toga što su CIFAR slike u boji, odnosno postoje
crveni, zeleni i plavi kanal.

Kako su dimenzije slika 32x32, mreža će uzimati ulaz veličine (32, 32, 3),
ili (3, 32, 32) u nekim drugim bibliotekama koje očekuju da prvo ide broj
kanala.

Kako bi definisali dimenziju ulaza, možemo podesiti imenovani argument
`input_shape` pri konstrukciji prvog sloja u `Sequential` modelu.


In [0]:
model = models.Sequential()
# Dodajemo konvolutivni sloj koji ima 32 filtera veličine 3x3 sa
# relu aktivacionom funkcijom.
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
# Dodajemo agregirajući sloj koji koristi funckiju maksimuma
# pri čemu je veličina filtera 2x2.
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Možemo da pogledamo naš model koristeći `summary` funkciju.

In [0]:
model.summary()

Možemo primetiti da broj kanala raste kako se ide dublje u mrežu.

Formula za izračunavanje dimenzija izlaza konvolucije i agregacije:

$$
O_w = \frac{W - K + 2P}{S} + 1
$$

$$
O_h = \frac{H - K + 2P}{S} + 1
$$

gde je:
- $O_w$: izlazna dimenzija - širina
- $O_h$: izlazna dimenzija - virina
- W: širina
- H: visina
- P: padding
- S: stride

Odnosno, pošto su najčešće širina i visina jednake (N = W = H), možemo reći:

$$
O = \frac{N - K + 2P}{S} + 1
$$


Na primer, prvi konvolutivni sloj koji smo dodali transformiše
(32, 32, 3) u (30, 30, 32).

- N = 32 = W = H
- H = 32
- P = 0
- S = 1
- K = 3

$$
O = \frac{32 - 3 + 2 \cdot 0}{1} + 1 = 32 - 3 + 0 + 1 = 30
$$



### Klasifikacioni model
U drugom delu našeg modela, želimo da napravimo klasifikacioni deo
koji će da omogući da se izvrši klasifikacija nad CIFAR skupom podataka.
Ovaj deo će dosta ličiti na prošle vežbe gde smo pravili klasifikacioni model
za Fashnion Mnist skup podataka.

Kako treuntno izlaz modela daje nešto oblika (4, 4, 64), neophodno je da
to transformišemo u vektor koji će se dati kao ulaz potpuno povezanoj
neuronskoj mreži koje će predstavljati klasifikator. Primetite da smo u primeru
za Fashion Mnist koristi `Flatten` da transformišemo sliku u vektor, a upravo
to ćemo učiniti i ovde.

Odnosno, Flatten((4, 4, 64)) biće vektor dimenzije 4x4x64 = 1024.

Kako skup ima 10 klasa, na izlazu modela ćemo staviti 10 neurona.


In [0]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Kompletna arhitektura modela.

In [0]:
model.summary()

### Obučavanje modela

In [0]:
epochs = 10
batch_size = 64
num_classes = 10

train_labels_cat = tf.keras.utils.to_categorical(train_labels, num_classes)
test_labels_cat = tf.keras.utils.to_categorical(test_labels, num_classes)

print(f'train_labels.shape={train_labels.shape}')
print(f'test_labels.shape={test_labels.shape}')
print(f'train_labels_cat.shape={train_labels_cat.shape}')
print(f'test_labels_cat.shape={test_labels_cat.shape}')

model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])


history = model.fit(train_images, train_labels_cat, epochs=epochs,
                    batch_size=64,  validation_data=(test_images, test_labels_cat))

### Evaluacija modela

In [0]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels_cat, verbose=2)

In [0]:
print(test_acc)

Naša mreža ja ostvarila tačnost od oko 70%, naravno, to može dosta bolje!
U narednom primeru ćemo videti neke od tehnika kako movo možemo da poboljšamo
korišćenjem augmentacije slika kao i korišćenjem regularizacije
izostavljanjem (eng. dropout).

DOMAĆI: Pokušajte sami da poboljšate ovu tačnost! Neke od ideja:
- Koristite dublji model
- Povećajte broj filtera
- Obučavajte mrežu duže
- Menjajte `batch_size`
- Koristite `dropout`


### Čuvanje i učitavanje mreže na disku

Rad u praksi sa neuronskim mrežama se retko realizuje tako što se jednom pokrene
trening za određeni broj epoha, sačeka da se dobije mreža, i potom se ona
koristi u daljem radu. Vrlo često u stvari imamo iterativni proces gde se mreža
trenira do neke tačke, uzme njena verzija i pusti u produkciju, ali se 
trening mreža nastavlja. To znači da će tokom treninga postojati razne
verzije mreže, i u smislu arhitekture, i u smislu težina.

Keras nam omogućava da čuvamo i učitavamo mreže na jednostavan način putem
funkcija `save` i `load`. Ovo ima ograničenja ukoliko se koriste
korisnički definisane funkcije greške i slojevi, što na ovom kursu
svakako nećemo raditi.

Na primer, prethodno
obučenu mrežu možemo sačuvati na disk (celokupan model) na sledeći
način:

In [12]:
! mkdir models

model_path = 'models/basic_network.h5'
model.save(model_path)

! ls models

mkdir: cannot create directory ‘models’: File exists
basic_network.h5


Ova datoteka se može negde sačuvati i iskoristiti da se instancira mreža.
Važno je napomenuti da `save` čuva arhitekturu mreže, težine mreže kao i
informacije koje su prosleđene tokom kompilacije mreže (`compile`).

Sada ćemo najpre postojeću mrežu (referenca `model`) da uništimo, a potom
da zamislimo da smo na drugom računaru i da treba da učitamo mrežu dostupnu na
putanji `models/basic_network.h5`.

In [0]:
backup_model = model
model = None

In [0]:
model = tf.keras.models.load_model(model_path)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)                6

In [16]:
test_loss, test_acc = model.evaluate(test_images,  test_labels_cat, verbose=2)

313/313 - 1s - loss: 0.8034 - accuracy: 0.7232


Možete primetiti da je u pitanju isti rezultat kao u prošloj ćeliji gde
smo evaluirali mrežu.

Više o čuvanju modela dostupno je na sledećoj [adresi](https://www.tensorflow.org/guide/keras/save_and_serialize).

### Čuvanje modela tokom obučavanja

Tokom obučavanja mreže, možda želimo da se sačuvaju razne verzije mreže jer nam
može biti korisno da rekonstruišemo neku verziju mreže.

Keras nudi `ModelCheckpoint` funkcionalnost (eng. callback) koja omogućava
da se tokom treninga sačuva trenutna verzija modela. Više informacija
dostupno je [ovde](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint).